In [61]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import numpy as np
from datetime import datetime
import re
import string
import pickle
import requests
import json

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [62]:
model = pickle.load(open('./old/modelSVC.pickle','rb'))
tfidf_vectorizer =  pickle.load(open('./old/tfidf_vectorizer.pickle','rb'))
APIurl = 'http://127.0.0.1:3000'
req = requests

/var/folders/ch/fxrhbppn0ds9grnhwp1j2qfm0000gn/T/ipykernel_7975/3466385359.py:1: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  model = pickle.load(open('./old/modelSVC.pickle','rb'))
/Users/qoisoctava/miniforge3/envs/pySA/lib/python3.9/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator SVC from version 1.1.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/var/folders/ch/fxrhbppn0ds9grnhwp1j2qfm0000gn/T/ipykernel_7975/3466385359.py:2: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  tfidf_vectorizer =  pickle.load(open('./old/tfidf_vectorizer.pickle','rb'))
/Users/qoisoctava/miniforge3/envs/pySA/lib/python3.9/s

In [63]:
def scraperKey(keyword,date_since,date_until):
    query = keyword+" lang:id until:"+str(date_until)+" since:"+str(date_since)
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    req.post('{}/progress/new'.format(APIurl),{'dateGet':now,'keyword':keyword,'dateSince':date_since,'dateUntil':date_until,'status':1,'source':'tw'} )
    #print(query)
    #print(datetime.now())
    #print("Sedang Mengumpulkan Data Twitter...")
    tweets = []
    while len(tweets) <= 20:
        for tweet in sntwitter.TwitterSearchScraper(query).get_items():
            tweets.append([datetime.now().date(), keyword, tweet.date, tweet.user.username, tweet.content, tweet.hashtags,tweet.mentionedUsers, tweet.likeCount, tweet.retweetCount, tweet.replyCount])
        
        
        
    df = pd.DataFrame(tweets, columns=['timestamp','keyword','date', 'user', 'tweet', 'hashtags', 'mentions', 'likeCount', 'retweetCount', 'replyCount'])
    return df,now


In [64]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # remove mentions
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # remove hashtag
    text = re.sub(r'RT[\s]', '', text) # remove RT
    text = re.sub(r"http\S+", '', text) # remove link
    text = re.sub(r'[0-9]+', '', text) # remove numbers
    text = text.encode("ascii", "ignore").decode() #remove emojis

    text = text.replace('\n', ' ') # replace new line into space
    text = text.translate(str.maketrans('', '', string.punctuation)) # remove all punctuations
    text = text.strip(' ') # remove characters space from both left and right text
    return text

def casefoldingText(text): # Converting all the characters in a text into lower case
    text = text.lower() 
    return text

def tokenizingText(text): # Tokenizing or splitting a string, text into a list of tokens
    text = word_tokenize(text) 
    return text

def filteringText(text): # Remove stopwors in a text
    listStopwords = set(stopwords.words('indonesian'))
    text = [w for w in text if not w in listStopwords]
    return text

def stemmingText(text): # Reducing a word to its word stem that affixes to suffixes and prefixes or to the roots of words
    #factory = StemmerFactory()
    from nltk.stem import PorterStemmer
    stemmer = PorterStemmer()
    text = [stemmer.stem(word) for word in text]
    return text

def toSentence(list_words): # Convert list of words into sentence
    sentence = ' '.join(word for word in list_words)
    return sentence

alay_dict = pd.read_csv('colloquial-indonesian-lexicon.csv', encoding='latin-1', header=None)
alay_dict = alay_dict.rename(columns={0: 'original', 1: 'replacement'})
alay_dict_map = dict(zip(alay_dict['original'], alay_dict['replacement']))

def normalize_alay(text):
 text = ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])
 text = re.sub(' +', ' ', text)
 return text



In [65]:
df, now = scraperKey('bandara yia','2023-01-13','2023-01-14')
topic = 'wisata jogja'

/var/folders/ch/fxrhbppn0ds9grnhwp1j2qfm0000gn/T/ipykernel_7975/1687530207.py:11: FutureWarning: content is deprecated, use rawContent instead
  tweets.append([datetime.now().date(), keyword, tweet.date, tweet.user.username, tweet.content, tweet.hashtags,tweet.mentionedUsers, tweet.likeCount, tweet.retweetCount, tweet.replyCount])


In [66]:
id = requests.get('{}/progress/id/{}'.format(APIurl,now)).json()[0]['id']
print(id)

29


In [67]:
def update(code):
    req.put('{}/progress/update'.format(APIurl), data={'id':id,'status': code})

In [68]:

if df.empty:
    update(404)
    print("Tidak ada data atau query keliru!")
else:
    df = df.drop_duplicates(subset=['tweet'])
    dataset = df
    update(2)
    #print(dataset)
    print("Sedang Membersihkan Data Twitter...")
    #gabungin hashtags
    # dataset['Hashtags'] = '-'.join(dataset['Hashtags'])
    print(datetime.now())
    print('cleaning text')
    dataset['text_clean'] = dataset['tweet'].apply(cleaningText)
    print(datetime.now())
    print('casefolding text')
    dataset['text_clean'] = dataset['text_clean'].apply(casefoldingText)
    dataset['text_clean'] = dataset['text_clean'].apply(normalize_alay)
    
    print(datetime.now())
    print('tokenizing text')
    dataset['text_preprocessed'] = dataset['text_clean'].apply(tokenizingText)
    print(datetime.now())
    print("filtering text")
    dataset['text_preprocessed'] = dataset['text_preprocessed'].apply(filteringText)
    print(datetime.now())
    print('stemming text')
    dataset['text_preprocessed'] = dataset['text_preprocessed'].apply(stemmingText)
    dataset["popularityScore"] = (dataset["likeCount"] + dataset["retweetCount"] + dataset["replyCount"])/3
    print("Selesai Membersihkan Data Twitter :)")
    print(datetime.now())

        ###################################
        ###      SENTIMENT ANALYSIS     ###
        ###################################

    print("Sedang Menganalisis Sentimen Publik...")
    update(3)

    print(datetime.now())
    # Make text preprocessed (tokenized) to untokenized with toSentence Function
    X = dataset['text_preprocessed'].apply(toSentence)
    X = tfidf_vectorizer.transform(X.values)
    
    # e = X.toarray()
    # #rumus mean
    # n = 0
    # mean = []
    # for i in e:
    #     a = sum(i.tolist())/5000
    #     mean.append(a)

    # #standar deviasi
    # n = 0
    # stdev = []
    # for i in e:
    #     s = np.std(i.tolist())
    #     stdev.append(s)

    # #max value
    # n = 0
    # maks = []
    # for i in e:
    #     a = max(i.tolist())
    #     maks.append(a)

    # #sum
    # n = 0
    # summ = []
    # for i in e:
    #     a = sum(i.tolist())
    #     summ.append(a)

    # #count feature
    # n = 0
    # count = []
    # for i in e:
    #     a = sum(map(lambda x : x != 0, i.tolist()))
    #     count.append(a)

    # extra_X = np.column_stack((e, np.array(mean), np.array(stdev), np.array(maks), np.array(summ), np.array(count)))

    # from scipy import sparse
    # extra_X = sparse.csr_matrix(extra_X)

    y_pred = model.predict(X)
    dataset['sentiment'] = y_pred

    polarity_decode = {0 : 'Negative', 1 : 'Neutral', 2 : 'Positive'}
    dataset['sentiment'] = dataset['sentiment'].map(polarity_decode)
    print("Selesai Menganalisis Sentimen Publik :)")

    print(datetime.now())

    dataset['date'] = pd.to_datetime(dataset['date']).dt.date
    dataset['date']=dataset['date'].astype(str)
        
    print(dataset.dtypes)

    

Sedang Membersihkan Data Twitter...
2023-01-14 17:26:04.600191
cleaning text
2023-01-14 17:26:04.600776
casefolding text
2023-01-14 17:26:04.601143
tokenizing text
2023-01-14 17:26:04.602080
filtering text
2023-01-14 17:26:04.604520
stemming text
Selesai Membersihkan Data Twitter :)
2023-01-14 17:26:04.606298
Sedang Menganalisis Sentimen Publik...
2023-01-14 17:26:04.608559
Selesai Menganalisis Sentimen Publik :)
2023-01-14 17:26:04.614020
timestamp             object
keyword               object
date                  object
user                  object
tweet                 object
hashtags              object
mentions              object
likeCount              int64
retweetCount           int64
replyCount             int64
text_clean            object
text_preprocessed     object
popularityScore      float64
sentiment             object
dtype: object


In [69]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 0 to 10
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   timestamp          11 non-null     object 
 1   keyword            11 non-null     object 
 2   date               11 non-null     object 
 3   user               11 non-null     object 
 4   tweet              11 non-null     object 
 5   hashtags           3 non-null      object 
 6   mentions           3 non-null      object 
 7   likeCount          11 non-null     int64  
 8   retweetCount       11 non-null     int64  
 9   replyCount         11 non-null     int64  
 10  text_clean         11 non-null     object 
 11  text_preprocessed  11 non-null     object 
 12  popularityScore    11 non-null     float64
 13  sentiment          11 non-null     object 
dtypes: float64(1), int64(3), object(10)
memory usage: 1.3+ KB


In [70]:
df = dataset[['timestamp', 'keyword', 'date', 'user', 'tweet', 'hashtags', 'mentions', 'likeCount', 'retweetCount', 'replyCount','popularityScore',  'sentiment']]
df

,timestamp,keyword,date,user,tweet,hashtags,mentions,likeCount,retweetCount,replyCount,popularityScore,sentiment
0,2023-01-14,bandara yia,2023-01-13,midfieldeer14,Ada yang mau kerja di bandara lur khusus cewek...,None,None,8,5,8,7.000000,Positive
1,2023-01-14,bandara yia,2023-01-13,infobmkgyia,"Hai #sobatbmkgyia,\nBerikut kami sampaikan Pra...","[sobatbmkgyia, infobmkg, infobmkgyia, prakiraa...",None,1,0,0,0.333333,Positive
2,2023-01-14,bandara yia,2023-01-13,yosanaa_,Kenapa bandara yia jauh banget jadinya harus b...,None,None,0,0,0,0.000000,Neutral
3,2023-01-14,bandara yia,2023-01-13,Minariiiing,Bandara. \nYIA Kulon Progo mantep sih. Viewnya...,None,None,0,0,0,0.000000,Positive
4,2023-01-14,bandara yia,2023-01-13,sewamobilmurah9,Ready city car matic manual\nBiar bisa sat set...,"[rentalmobiljogja, sewamobiljogja, rentalmobil...",None,0,0,0,0.000000,Positive
5,2023-01-14,bandara yia,2023-01-13,annipusphyta,Parkir mahal di Diy jatuh pada parkir vip band...,None,None,0,0,0,0.000000,Positive
6,2023-01-14,bandara yia,2023-01-13,stefanaezer,"@KompasTV Dari bukan Jakarta Jogja, tp Tangera...",None,[https://twitter.com/KompasTV],0,0,0,0.000000,Neutral
7,2023-01-14,bandara yia,2023-01-13,CecanUtami,berapa harga tiket kereta bandara yia,None,None,0,0,0,0.000000,Neutral
8,2023-01-14,bandara yia,2023-01-13,KAI121,@zeuschlad Selamat pagi Kak. Perjalanan KA Ban...,None,"[https://twitter.com/zeuschlad, https://twitte...",0,0,1,0.333333,Positive
9,2023-01-14,bandara yia,2023-01-13,jhnhpt,@UGM_FESS dari Jakarta ke Bandara juga perlu k...,None,[https://twitter.com/UGM_FESS],0,0,0,0.000000,Neutral


In [71]:
# cols = ['Date','User','Tweet','Hashtags','Like','Retweet','Reply','Popularity Score','Result Prediction','keyword','timestamp']
# df = cleanData[cols]
# ncols = ['date','user','tweet','hashtags','like','retweet','reply','popularityScore','sentiment','keyword','timestamp']
# send

In [72]:
df.fillna('0')

,timestamp,keyword,date,user,tweet,hashtags,mentions,likeCount,retweetCount,replyCount,popularityScore,sentiment
0,2023-01-14,bandara yia,2023-01-13,midfieldeer14,Ada yang mau kerja di bandara lur khusus cewek...,0,0,8,5,8,7.000000,Positive
1,2023-01-14,bandara yia,2023-01-13,infobmkgyia,"Hai #sobatbmkgyia,\nBerikut kami sampaikan Pra...","[sobatbmkgyia, infobmkg, infobmkgyia, prakiraa...",0,1,0,0,0.333333,Positive
2,2023-01-14,bandara yia,2023-01-13,yosanaa_,Kenapa bandara yia jauh banget jadinya harus b...,0,0,0,0,0,0.000000,Neutral
3,2023-01-14,bandara yia,2023-01-13,Minariiiing,Bandara. \nYIA Kulon Progo mantep sih. Viewnya...,0,0,0,0,0,0.000000,Positive
4,2023-01-14,bandara yia,2023-01-13,sewamobilmurah9,Ready city car matic manual\nBiar bisa sat set...,"[rentalmobiljogja, sewamobiljogja, rentalmobil...",0,0,0,0,0.000000,Positive
5,2023-01-14,bandara yia,2023-01-13,annipusphyta,Parkir mahal di Diy jatuh pada parkir vip band...,0,0,0,0,0,0.000000,Positive
6,2023-01-14,bandara yia,2023-01-13,stefanaezer,"@KompasTV Dari bukan Jakarta Jogja, tp Tangera...",0,[https://twitter.com/KompasTV],0,0,0,0.000000,Neutral
7,2023-01-14,bandara yia,2023-01-13,CecanUtami,berapa harga tiket kereta bandara yia,0,0,0,0,0,0.000000,Neutral
8,2023-01-14,bandara yia,2023-01-13,KAI121,@zeuschlad Selamat pagi Kak. Perjalanan KA Ban...,0,"[https://twitter.com/zeuschlad, https://twitte...",0,0,1,0.333333,Positive
9,2023-01-14,bandara yia,2023-01-13,jhnhpt,@UGM_FESS dari Jakarta ke Bandara juga perlu k...,0,[https://twitter.com/UGM_FESS],0,0,0,0.000000,Neutral


In [73]:
# df[['timestamp','date']] = df[['timestamp','date']].astype(str)

In [74]:
# api = df.to_json("./test_data/json.json",orient='records')
apid = df.to_dict(orient='records')

In [75]:




# x = requests.post(url, json = api)

# print(x.text)

In [76]:


for i in apid:
    dicts ={
        'dateGet':i['timestamp'],
        'keyword':i['keyword'],
        'contentDate':i['date'],
        'username':i['user'],
        'tweet':i['tweet'],
        'hashtags':i['hashtags'],
        'mentions':i['mentions'],
        'likeCount':i['likeCount'],
        'retweetCount':i['retweetCount'],
        'replyCount':i['replyCount'],
        'popularityScore':i['popularityScore'],
        'sentiment':i['sentiment'],
        'topic':topic
        }
    # print(dicts)
    c = requests.post('{}/post'.format(APIurl),dicts )
    print(c)

update(4)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [77]:
# cleanData[['Tweet', 'Result Prediction', 'text_preprocessed']].to_csv('pemilu.csv')
  
